This notebook demonstrates how to predict IC50 from BindingDB data using the digital-cell model, as well as giving suggestions for how to enhance the performance of the model beyond that shown here.

In [3]:
import os
import digitalcell

from DeepPurpose import utils, dataset, CompoundPred
from DeepPurpose import DTI as models
import warnings
warnings.filterwarnings("ignore")

# Data Importing

The entirety of BindingDB is saved as a .tsv file. The function `digital-cell.process_BindingDB_omic` loads it into a Pandas dataframe.

In [4]:
data_path = './data//BindingDB_All.tsv'
df, X_drugs, X_targets, y = digitalcell.process_BindingDB_omic(path = data_path, temp_ph = True)

Loading Dataset from path...


b'Skipping line 772572: expected 193 fields, saw 205\nSkipping line 772598: expected 193 fields, saw 205\n'
b'Skipping line 805291: expected 193 fields, saw 205\n'
b'Skipping line 827961: expected 193 fields, saw 265\n'
b'Skipping line 1231688: expected 193 fields, saw 241\n'
b'Skipping line 1345591: expected 193 fields, saw 241\nSkipping line 1345592: expected 193 fields, saw 241\nSkipping line 1345593: expected 193 fields, saw 241\nSkipping line 1345594: expected 193 fields, saw 241\nSkipping line 1345595: expected 193 fields, saw 241\nSkipping line 1345596: expected 193 fields, saw 241\nSkipping line 1345597: expected 193 fields, saw 241\nSkipping line 1345598: expected 193 fields, saw 241\nSkipping line 1345599: expected 193 fields, saw 241\n'
b'Skipping line 1358864: expected 193 fields, saw 205\n'
b'Skipping line 1378087: expected 193 fields, saw 241\nSkipping line 1378088: expected 193 fields, saw 241\nSkipping line 1378089: expected 193 fields, saw 241\nSkipping line 1378090: e

Beginning Processing...
There are 93336 drug target pairs.


The cell below just shows what the data looks like.

In [3]:
df.head()

,ID,InChI,SMILES,PubChem_ID,UniProt_ID,Organism,Target Sequence,Kd,IC50,Ki,EC50,kon,koff,pH,Temp,pIC50
180,181,InChI=1/C31H51N5O5/c1-20(2)28(32-22(5)37)30(40...,CC(C)[C@H](NC(C)=O)C(=O)N[C@@H](Cc1ccccc1)[C@@...,65023.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,8.5,NaN,NaN,NaN,NaN,6.0,37.0,8.065502
181,182,InChI=1/C33H55N5O7/c1-7-44-32(42)35-28(22(3)4)...,CCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1)[...,461984.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,177.0,NaN,NaN,NaN,NaN,6.0,37.0,6.751781
183,184,InChI=1/C35H59N5O9/c1-24(2)30(37-34(44)48-19-1...,COCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccccc1...,461988.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,164.0,NaN,NaN,NaN,NaN,6.0,37.0,6.784891
184,185,InChI=1/C39H67N5O11/c1-28(2)34(41-38(48)54-23-...,COCCOCCOC(=O)N[C@@H](C(C)C)C(=O)N[C@@H](Cc1ccc...,461990.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,67.0,NaN,NaN,NaN,NaN,6.0,37.0,7.173277
185,186,InChI=1/C38H51N7O7/c1-24(2)34(43-38(51)52-3)37...,COC(=O)N[C@@H](C(C)C)C(=O)NN(C[C@H](O)[C@H](Cc...,461985.0,NaN,Human immunodeficiency virus 1,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,NaN,27.0,NaN,NaN,NaN,NaN,6.0,37.0,7.567031


# Drug and target encoding

DeepPurpose supports several possible encodings for drugs and targets. I suspect choosing and possibly combining these will be key for feature engineering and model performance. Right now I'm using the Morgan Extended-Connectivity Fingerprints encoding for drugs and the conjoint triad features encoding for targets because they're not computationally intensive. I would like to find a way to allow models to take PaDEL features as drug encodings, which is what the gastric cancer paper did. They also encoded targets in terms of mutation position, which may not be feasible for all targets, but is probably worth looking into.

In [4]:
drug_encoding, target_encoding = 'Morgan', 'Conjoint_triad'

The following cell splits the data into training and testing sets. I think these numbers are probably fine, but maybe there's a way to improve on them. One thing to note is that for some reason RDKit (the biochemistry library) has trouble translating some of the SMILES data (~500 drugs) into Morgan format. That's only a small fraction of the drugs in the training data, though.

In [5]:
train, val, test = utils.data_process(X_drugs, X_targets, y, 
                                drug_encoding, target_encoding, 
                                split_method='random',frac=[0.7,0.1,0.2])

Drug Target Interaction Prediction Mode...
in total: 93336 drug-target pairs
encoding drug...
unique drugs: 63535
rdkit not found this smiles for morgan: CSc1ccc(cc1)C1=C(C=C[N]([O-])=C1)[C@@H]1CCC(F)(F)C[C@H]1C(=O)NCC#N convert to all 0 features
rdkit not found this smiles for morgan: O=C1NC(=O)c2c1c1c3ccccc3n3[Ru](C#[O])[n+]4cccc2c4c13 convert to all 0 features
rdkit not found this smiles for morgan: CN1C(=O)c2c(C1=O)c1cc(F)c[n+]3[Ru](C#[O])n4c5ccc(O)cc5c2c4c13 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(CC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(CC[N@@]23CC(=O)O[Re]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles for morgan: CN1C=C[N]2=C1C[N]1(CCc3ccc(SOON)cc3)CC3=[N](C=CN3C)[Re+]21 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(NC(=S)NCCCCCCCC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features

rdkit not found this smiles for morgan: Clc1cccc(Cl)c1NC(=O)N1CCN(CC1)c1ccc(cc1)-c1[nH]cnn1CCc1ccccc1 convert to all 0 features
rdkit not found this smiles for morgan: Clc1cccc(Cl)c1NC(=O)N1CCN(CC1)c1ccc(cc1)-c1ncnn1 convert to all 0 features
rdkit not found this smiles for morgan: Cc1ccc(cc1)-c1nc([nH]o1)-c1ccc(cc1)N1CCN(CC1)C(=O)Nc1ccccc1Cl convert to all 0 features
rdkit not found this smiles for morgan: Cn1cc(ccc1C(N)=O)S(=O)(=O)c1ccc(CN\C(Nc2ccncc2)=N/C#N)cc1 convert to all 0 features
rdkit not found this smiles for morgan: Cc1cc(nn1)-c1cc(C(=O)Nc2ccc(F)cn2)c2ncnn2c1 convert to all 0 features
rdkit not found this smiles for morgan: C[N]1=C(OC(=C1)c1cccc(Nc2nccc(n2)-c2cccnc2)c1)N1CCNC(=O)C11CC1 convert to all 0 features
rdkit not found this smiles for morgan: Cc1ccnc(Nc2cccc(C3=C[N](C)=C(O3)N3CCNC(=O)C33CC3)c2C)n1 convert to all 0 features
rdkit not found this smiles for morgan: CN1C=[N](C)C=C1c1ccc(C[C@H](NC(=O)[C@H]2N[C@@H]3CC[C@H]2C3)C#N)c(F)c1 convert to all 0 features
rdkit no

# Model configuration

DeepPurpose's model configuration utility is a wrapper for generating neural networks using PyTorch. The list of options for hyperparameters is [here](https://github.com/kexinhuang12345/DeepPurpose/blob/e169e2f550694145077bb2af95a4031abe400a77/DeepPurpose/utils.py#L486). Several types of model architecture are supported including CNNs, RNNs, MPNNs, MLPs, and transformers. I think there is a lot of potential work to be done on hyperparameter optimization here. The hyperparameters used below are suggested defaults that aren't too computationally intensive; they produce a 3 layer MPNN. I would also like to incorporate some models that don't utilize deep learning, as I think with proper input feature engineering we may be able to get good performance with an SVM or ridge regressor, and it would be nice to have a more interpretable model for comparison. 

In [6]:
config = utils.generate_config(drug_encoding = drug_encoding, 
                               target_encoding = target_encoding,
                         cls_hidden_dims = [1024,1024,512], 
                         train_epoch = 3, 
                         LR = 0.001, 
                         batch_size = 128,
                         hidden_dim_drug = 128,
                         mpnn_hidden_size = 128,
                         mpnn_depth = 3
                        )

In [7]:
model = models.model_initialize(**config)

# Model training and loading

Using the hyperparameters above, the model takes about 20 minutes on a GPU (1.5 hours on a laptop CPU) to train on BindingDB. For demo purposes I'm going to just load a model I trained earlier today, but I used the exact same code as above.

In [8]:
#run this if you want to train a new one
model.train(train, val, test, verbose = True)
model.save_model('./model-10-11')

Let's use CPU/s!
--- Data Preparation ---
--- Go for Training ---
Training at Epoch 1 iteration 0 with loss 50.2582. Total time 0.0 hours
Training at Epoch 1 iteration 100 with loss 1.57474. Total time 0.00638 hours


KeyboardInterrupt: 

In [3]:
#run this if you want to use my trained one
model = models.model_pretrained(path_dir = './model-9-24')

# Model validation

The following code is built in to the DeepPurpose `train` method, I've just pulled it out so I can grab the dataset that was set aside for validation during the data processing step. 

In [13]:
import torch
from torch.utils import data

params = {'batch_size': config['batch_size'],
    'shuffle': True,
    'num_workers': config['num_workers'],
    'drop_last': False}

validation_generator = data.DataLoader(utils.data_process_loader(val.index.values, val.Label.values, val, **config), **params)

The available performance metrics are ROC-AUC, PR-AUC, F1, cross-entropy loss, MSE, Pearson Correlation with p-value, and Concordance Index. I'm displaying the ROC-AUC here for comparison to the DeepIC50 paper. It's not at their level yet, but it's also not bad for a first try?

TODO: graph ROC curve, also why does test_ return "logits"

In [14]:
model.binary = False
auc, auprc, f1, loss, logits = models.DBTA.test_(model, validation_generator, model.model, test=True)
print(auc)

0.8197822689413422


# Model usage

To predict IC50, run data_process to create a dataset consisting of a single drug-target pair, and then run `predict`, which is just a wrapper on `test_`. The output is in pIC50. 

TODO: why do I need to include `y` as input?

In [28]:
X_drug = ['CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)N']
X_target = ['MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAPVRQQPKVQTTPPPAVQGQKVGSLTPPSSPKTQRAGHRRILSDVTHSAVFGVPASKSTQLLQAAAAEASLNKSKSATTTPSGSPRTSQQNVYNPSEGSTWNPFDDDNFSKLTAEELLNKDFAKLGEGKHPEKLGGSAESLIPGFQSTQGDAFATTSFSAGTAEKRKGGQTVDSGLPLLSVSDPFIPLQVPDAPEKLIEGLKSPDTSLLLPDLLPMTDPFGSTSDAVIEKADVAVESLIPGLEPPVPQRLPSQTESVTSNRTDSLTGEDSLLDCSLLSNPTTDLLEEFAPTAISAPVHKAAEDSNLISGFDVPEGSDKVAEDEFDPIPVLITKNPQGGHSRNSSGSSESSLPNLARSLLLVDQLIDL']
X_pred = utils.data_process(X_drug, X_target, y, 
                                drug_encoding, target_encoding, 
                                split_method='no_split')
y_pred = model.predict(X_pred)
print('The predicted score is ' + str(y_pred))

Drug Target Interaction Prediction Mode...
in total: 1 drug-target pairs
encoding drug...
unique drugs: 1
encoding protein...
unique target sequence: 1
splitting dataset...
do not do train/test split on the data for already splitted data
predicting...
The predicted score is [7.395412921905518]


# Part 2: Random Forest

## Data importing

There are a number of supported encodings; calling `digitalcell.feature_select` without specifying runs ones that have low overhead and perform well in the classifier.

TODO: import support for other datasets (incl conversion to SMILES), list of supported encodings

In [5]:
df_data = digitalcell.feature_select(df)

Encoding:  drug2emb_encoder
Elapsed time:  29.909013509750366
Encoding:  smiles2daylight
rdkit not found this smiles: CSc1ccc(cc1)C1=C(C=C[N]([O-])=C1)[C@@H]1CCC(F)(F)C[C@H]1C(=O)NCC#N convert to all 0 features
rdkit not found this smiles: O=C1NC(=O)c2c1c1c3ccccc3n3[Ru](C#[O])[n+]4cccc2c4c13 convert to all 0 features
rdkit not found this smiles: CN1C(=O)c2c(C1=O)c1cc(F)c[n+]3[Ru](C#[O])n4c5ccc(O)cc5c2c4c13 convert to all 0 features
rdkit not found this smiles: NOOSc1ccc(CC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles: NOOSc1ccc(CC[N@@]23CC(=O)O[Re]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles: CN1C=C[N]2=C1C[N]1(CCc3ccc(SOON)cc3)CC3=[N](C=CN3C)[Re+]21 convert to all 0 features
rdkit not found this smiles: NOOSc1ccc(NC(=S)NCCCCCCCC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles: NOOSc1ccc(NC(=S)NCCOCCOCC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 

rdkit not found this smiles: Cc1cc(nn1)-c1cc(C(=O)Nc2ccc(F)cn2)c2ncnn2c1 convert to all 0 features
rdkit not found this smiles: C[N]1=C(OC(=C1)c1cccc(Nc2nccc(n2)-c2cccnc2)c1)N1CCNC(=O)C11CC1 convert to all 0 features
rdkit not found this smiles: Cc1ccnc(Nc2cccc(C3=C[N](C)=C(O3)N3CCNC(=O)C33CC3)c2C)n1 convert to all 0 features
rdkit not found this smiles: CN1C=[N](C)C=C1c1ccc(C[C@H](NC(=O)[C@H]2N[C@@H]3CC[C@H]2C3)C#N)c(F)c1 convert to all 0 features
rdkit not found this smiles: Oc1cnc(Cn2cnc(c(Oc3cc(Cl)cc(c3)C#N)c2=O)C(F)(F)F)c[nH]1 convert to all 0 features
Elapsed time:  193.77500343322754
Encoding:  CalculateConjointTriad
Elapsed time:  194.86800026893616
Encoding:  protein2emb_encoder
Elapsed time:  228.59901332855225


If you want to include entries that don't have temp and pH:

TODO: better imputer, save raw data

If you only want entries that include temp and pH:

TODO: improve this

## Encodings for use with RF

In [6]:
model = models.model_pretrained(path_dir = './model-9-24')
X_pred = utils.data_process(X_drugs, X_targets, y, 
                                model.drug_encoding, model.target_encoding, 
                                split_method='no_split')

Drug Target Interaction Prediction Mode...
in total: 93336 drug-target pairs
encoding drug...
unique drugs: 63535
rdkit not found this smiles for morgan: CSc1ccc(cc1)C1=C(C=C[N]([O-])=C1)[C@@H]1CCC(F)(F)C[C@H]1C(=O)NCC#N convert to all 0 features
rdkit not found this smiles for morgan: O=C1NC(=O)c2c1c1c3ccccc3n3[Ru](C#[O])[n+]4cccc2c4c13 convert to all 0 features
rdkit not found this smiles for morgan: CN1C(=O)c2c(C1=O)c1cc(F)c[n+]3[Ru](C#[O])n4c5ccc(O)cc5c2c4c13 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(CC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(CC[N@@]23CC(=O)O[Re]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features
rdkit not found this smiles for morgan: CN1C=C[N]2=C1C[N]1(CCc3ccc(SOON)cc3)CC3=[N](C=CN3C)[Re+]21 convert to all 0 features
rdkit not found this smiles for morgan: NOOSc1ccc(NC(=S)NCCCCCCCC[N]23CC4=CC=CC=[N]4[Re+]2[N]2=C(C3)C=CC=C2)cc1 convert to all 0 features

rdkit not found this smiles for morgan: Clc1cccc(Cl)c1NC(=O)N1CCN(CC1)c1ccc(cc1)-c1[nH]cnn1CCc1ccccc1 convert to all 0 features
rdkit not found this smiles for morgan: Clc1cccc(Cl)c1NC(=O)N1CCN(CC1)c1ccc(cc1)-c1ncnn1 convert to all 0 features
rdkit not found this smiles for morgan: Cc1ccc(cc1)-c1nc([nH]o1)-c1ccc(cc1)N1CCN(CC1)C(=O)Nc1ccccc1Cl convert to all 0 features
rdkit not found this smiles for morgan: Cn1cc(ccc1C(N)=O)S(=O)(=O)c1ccc(CN\C(Nc2ccncc2)=N/C#N)cc1 convert to all 0 features
rdkit not found this smiles for morgan: Cc1cc(nn1)-c1cc(C(=O)Nc2ccc(F)cn2)c2ncnn2c1 convert to all 0 features
rdkit not found this smiles for morgan: C[N]1=C(OC(=C1)c1cccc(Nc2nccc(n2)-c2cccnc2)c1)N1CCNC(=O)C11CC1 convert to all 0 features
rdkit not found this smiles for morgan: Cc1ccnc(Nc2cccc(C3=C[N](C)=C(O3)N3CCNC(=O)C33CC3)c2C)n1 convert to all 0 features
rdkit not found this smiles for morgan: CN1C=[N](C)C=C1c1ccc(C[C@H](NC(=O)[C@H]2N[C@@H]3CC[C@H]2C3)C#N)c(F)c1 convert to all 0 features
rdkit no

In [7]:
first_pass = model.predict(X_pred)

predicting...


In [9]:
digitalcell.data_process_omic(df_data, first_pass)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [33]:
import pandas as pd

#cat_list = pd.get_dummies(df_data['Organism'], prefix='var')
#df_data=df_data.join(cat_list)
    
discard=['SMILES','Target Sequence','Organism','IC50','pIC50','ID','InChI','PubChem_ID','UniProt_ID','Kd','Ki','EC50','kon','koff']
df_vars=df_data.columns.values.tolist()
to_keep=[i for i in df_vars if i not in discard]
X=df_data[to_keep]

#idxlist = df_data.index.astype(int)
#first_pass = first_pass[idxlist]
X[len(X.columns)] = first_pass

#Z = np.empty(shape=[len(X),len(flattener(X.iloc[0]))])
#for n in range(len(X)):
#    Z[n] = flattener(X.iloc[n])

#TODO: allow adjusting this
tag = np.random.binomial(n=1, p=.8, size=len(X)) == 1

# assign True indices to idx1 and False indices to index 2
idx = np.array( range( len(X) ) )
idx1, idx2 = idx[ tag ], idx[ np.logical_not( tag ) ]

# sample from idx1 and idx2
#TODO: allow adjusting this
i1, i2 = np.random.choice( idx1, size=10000 ), np.random.choice( idx2, size=10000 )

In [37]:
Z = np.empty(shape=[len(X),len(digitalcell.flattener(X.iloc[0]))])
for n in range(len(X)):
    Z[n] = digitalcell.flattener(X.iloc[n])

In [38]:
X_train = Z[idx1]
X_test = Z[idx2]
y_train =df_data['pIC50'].iloc[idx1].to_numpy(dtype = object)
y_test = df_data['pIC50'].iloc[idx2].to_numpy(dtype = object)

test_list = pd.isnull(y_train)
res = [i for i, val in enumerate(test_list) if val] 
y_train = np.delete(y_train,res,0)
X_train = np.delete(X_train,res,0)

test_list = pd.isnull(y_test)
res = [i for i, val in enumerate(test_list) if val] 
y_test = np.delete(y_test,res,0)
X_test = np.delete(X_test,res,0)

In [44]:
model2 = GBoostModel(
    lower_alpha=0.1, upper_alpha=0.9, n_estimators=10, org_list = cat_list)

In [45]:
# Fit and make predictions
model2.fit(X_train, y_train)

Calculating estimates...
Getting lower bound...
Getting upper bound...


In [48]:
predictions = pd.DataFrame()
predictions["lower"] = model2.lower_model.predict(Z)
predictions["mid"] = model2.mid_model.predict(Z)
predictions["upper"] = model2.upper_model.predict(Z)
model2.predictions = predictions

In [51]:
X_drug = ['CC1=C2C=C(C=CC2=NN1)C3=CC(=CN=C3)OCC(CC4=CC=CC=C4)N']
X_target = ['MKKFFDSRREQGGSGLGSGSSGGGGSTSGLGSGYIGRVFGIGRQQVTVDEVLAEGGFAIVFLVRTSNGMKCALKRMFVNNEHDLQVCKREIQIMRDLSGHKNIVGYIDSSINNVSSGDVWEVLILMDFCRGGQVVNLMNQRLQTGFTENEVLQIFCDTCEAVARLHQCKTPIIHRDLKVENILLHDRGHYVLCDFGSATNKFQNPQTEGVNAVEDEIKKYTTLSYRAPEMVNLYSGKIITTKADIWALGCLLYKLCYFTLPFGESQVAICDGNFTIPDNSRYSQDMHCLIRYMLEPDPDKRPDIYQVSYFSFKLLKKECPIPNVQNSPIPAKLPEPVKASEAAAKKTQPKARLTDPIPTTETSIAPRQRPKAGQTQPNPGILPIQPALTPRKRATVQPPPQAAGSSNQPGLLASVPQPKPQAPPSQPLPQTQAKQPQAPPTPQQTPSTQAQGLPAQAQATPQHQQQLFLKQQQQQQQPPPAQQQPAGTFYQQQQAQTQQFQAVHPATQKPAIAQFPVVSQGGSQQQLMQNFYQQQQQQQQQQQQQQLATALHQQQLMTQQAALQQKPTMAAGQQPQPQPAAAPQPAPAQEPAIQAPVRQQPKVQTTPPPAVQGQKVGSLTPPSSPKTQRAGHRRILSDVTHSAVFGVPASKSTQLLQAAAAEASLNKSKSATTTPSGSPRTSQQNVYNPSEGSTWNPFDDDNFSKLTAEELLNKDFAKLGEGKHPEKLGGSAESLIPGFQSTQGDAFATTSFSAGTAEKRKGGQTVDSGLPLLSVSDPFIPLQVPDAPEKLIEGLKSPDTSLLLPDLLPMTDPFGSTSDAVIEKADVAVESLIPGLEPPVPQRLPSQTESVTSNRTDSLTGEDSLLDCSLLSNPTTDLLEEFAPTAISAPVHKAAEDSNLISGFDVPEGSDKVAEDEFDPIPVLITKNPQGGHSRNSSGSSESSLPNLARSLLLVDQLIDL']

In [66]:
test_1 = utils.data_process(X_drug, X_target, y, 
                                model.drug_encoding, model.target_encoding, 
                                split_method='no_split')
test_2 = model.predict(test_1)
test_3 = predict_drug(X_drug,X_target,test_2)

Drug Target Interaction Prediction Mode...
in total: 1 drug-target pairs
encoding drug...
unique drugs: 1
encoding protein...
unique target sequence: 1
splitting dataset...
do not do train/test split on the data for already splitted data
predicting...
Encoding:  drug2emb_encoder
Elapsed time:  0.0010001659393310547
Encoding:  smiles2daylight
Elapsed time:  0.00500178337097168
Encoding:  CalculateConjointTriad
Elapsed time:  0.00800013542175293
Encoding:  protein2emb_encoder
Elapsed time:  0.009000062942504883


In [67]:
test_3

,pH,Temp,drug2emb_encoder,smiles2daylight,CalculateConjointTriad,protein2emb_encoder,var_Abelson murine leukemia virus,var_Agaricus bisporus,var_Aspergillus fumigatiaffinis,var_Avian sarcoma virus,...,var_Torpedo marmorata,var_Trichomonas vaginalis G3,var_Trypanosoma brucei,var_Trypanosoma brucei brucei,var_Trypanosoma cruzi,var_Vibrio proteolyticus,var_Xenopus laevis,var_Yersinia pestis,var_[Bacteroides] pectinophilus ATCC 43243,86
0,7,35,"([800, 122, 248, 282, 623, 272, 1256, 2210, 91...","[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[8, 7, 10, 10, 2, 5, 0, 13, 9, 4, 8, 3, 7, 2, ...","([839, 167, 106, 61, 277, 447, 291, 2515, 32, ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.395413


In [73]:
digitalcell.flattener(np.asarray(test_3))

[7,
 35,
 800,
 122,
 248,
 282,
 623,
 272,
 1256,
 2210,
 91,
 85,
 109,
 119,
 80,
 8,
 282,
 861,
 209,
 19,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 

In [69]:
X

,pH,Temp,drug2emb_encoder,smiles2daylight,CalculateConjointTriad,protein2emb_encoder,var_Abelson murine leukemia virus,var_Agaricus bisporus,var_Aspergillus fumigatiaffinis,var_Avian sarcoma virus,...,var_Torpedo marmorata,var_Trichomonas vaginalis G3,var_Trypanosoma brucei,var_Trypanosoma brucei brucei,var_Trypanosoma cruzi,var_Vibrio proteolyticus,var_Xenopus laevis,var_Yersinia pestis,var_[Bacteroides] pectinophilus ATCC 43243,86
180,6.0,37.0,"([2266, 117, 72, 124, 339, 295, 186, 277, 1880...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.148507
181,6.0,37.0,"([240, 339, 1416, 295, 186, 277, 1880, 1436, 1...","[0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.638949
183,6.0,37.0,"([2227, 339, 1416, 295, 186, 277, 1880, 1436, ...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,6.822081
184,6.0,37.0,"([867, 486, 339, 1416, 295, 186, 277, 1880, 14...","[0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.156025
185,6.0,37.0,"([92, 2346, 199, 179, 1397, 206, 763, 124, 81,...","[1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 3, 0, 1, 0, 1, 0, 3, 2, 1, 0, 0, 1, 0, 1, ...","([14, 212, 35, 2864, 47, 69, 86, 497, 3636, 21...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.397277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1062791,7.5,22.0,"([277, 87, 168, 285, 557, 468, 171, 70, 124, 6...","[1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, ...","[3, 4, 1, 3, 1, 1, 0, 4, 4, 4, 2, 1, 1, 0, 2, ...","([300, 23, 326, 61, 23, 188, 26, 194, 1811, 28...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.843153
1751877,2.5,25.0,"([622, 182, 107, 32, 361, 122, 820, 1885, 583,...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[7, 10, 4, 0, 2, 5, 3, 8, 14, 2, 2, 0, 3, 1, 5...","([132, 179, 23, 865, 73, 798, 434, 115, 23, 18...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.641105
1751879,2.5,25.0,"([622, 182, 107, 32, 361, 122, 820, 1885, 583,...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[7, 10, 4, 0, 2, 5, 3, 8, 14, 2, 2, 0, 3, 1, 5...","([132, 179, 23, 865, 73, 798, 434, 115, 23, 18...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,6.392223
1751880,2.5,25.0,"([1953, 1644, 122, 820, 1885, 186, 1287, 99, 1...","[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","[7, 10, 4, 0, 2, 5, 3, 8, 14, 2, 2, 0, 3, 1, 5...","([132, 179, 23, 865, 73, 798, 434, 115, 23, 18...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.267920


In [68]:
    Z = np.empty(len(flattener(tes.iloc[0]))])
    for n in range(len(X)):
        Z[n] = flattener(X.iloc[n])

array([[   6.        ,   37.        , 2227.        , ...,    0.        ,
           0.        ,    6.82208061],
       [   6.        ,   37.        ,  240.        , ...,    0.        ,
           0.        ,    7.22141647],
       [   6.        ,   37.        , 2227.        , ...,    0.        ,
           0.        ,    7.66812134],
       ...,
       [   7.5       ,   25.        ,   43.        , ...,    0.        ,
           0.        ,    7.30627823],
       [   2.5       ,   25.        , 1953.        , ...,    0.        ,
           0.        ,    7.26792049],
       [   2.5       ,   25.        ,  622.        , ...,    0.        ,
           0.        ,    8.0207901 ]])

In [65]:
def predict_drug(X_drug, X_target, first_pass, temp = 35, pH = 7, org = None):
        """
        Predict with all 3 models 
        
        :param X: test features
        :param y: test targets
        :return predictions: dataframe of predictions
        
        TODO: parallelize this code across processors
        """
        df_data = pd.DataFrame()
        df_data['SMILES'] = X_drug
        df_data['Target Sequence'] = X_target
        if model2.temp_ph:
            df_data['pH'] = pH
            df_data['Temp'] = temp
        
        df_data = digitalcell.feature_select(df_data, model2.drug_func_list, model2.prot_func_list)
        df_data=df_data.join(model2.org_list)
        if org is not None:
            df_data['var_'+org] = 1
            
        discard=['SMILES','Target Sequence','Organism','IC50','pIC50','ID','InChI','PubChem_ID','UniProt_ID','Kd','Ki','EC50','kon','koff']
        df_vars=df_data.columns.values.tolist()
        to_keep=[i for i in df_vars if i not in discard]
        X=df_data[to_keep]
        X[len(X.columns)] = first_pass
        return X

In [ ]:
filename = 'model_10_12.sav'
pickle.dump(model2, open(filename, 'wb'))

TODO: save moel, usage (incl for imported data), confidence intervals, recommended drugs per target, data visualization, boruta or whatever

In [42]:
from sklearn.base import BaseEstimator
from DeepPurpose.utils import *
from DeepPurpose.model_helper import Encoder_MultipleLayers, Embeddings        
from DeepPurpose.encoders import *
from DeepPurpose import DTI

from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

class GBoostModel(BaseEstimator):
    """
    Adapted from https://github.com/WillKoehrsen/Data-Analysis/blob/master/prediction-intervals/prediction_intervals.ipynb
    Model that produces prediction intervals with a Scikit-Learn inteface
    
    :param lower_alpha: lower quantile for prediction, default=0.1
    :param upper_alpha: upper quantile for prediction, default=0.9
    :param **kwargs: additional keyword arguments for creating a GradientBoostingRegressor model
    """

    def __init__(self, lower_alpha=0.1, upper_alpha=0.9, drug_func_list = [drug2emb_encoder,smiles2daylight], 
                    prot_func_list = [CalculateConjointTriad, protein2emb_encoder], temp_ph = True, org_list = None,
                    n_estimators = 10, **kwargs):
        self.lower_alpha = lower_alpha
        self.upper_alpha = upper_alpha

        self.drug_func_list = drug_func_list
        self.prot_func_list = prot_func_list
        self.temp_ph = temp_ph
        if org_list is not None:
            self.org_list = org_list
        else:
            self.org_list = []
            #TODO: figure out a way to do a default here
            
        # Three separate models
        self.lower_model = GradientBoostingRegressor(loss="quantile", alpha=self.lower_alpha, n_estimators = n_estimators, **kwargs)
        self.mid_model = GradientBoostingRegressor(loss="ls", n_estimators = n_estimators, **kwargs)
        self.upper_model = GradientBoostingRegressor(loss="quantile", alpha=self.upper_alpha, n_estimators = n_estimators, **kwargs)
        self.predictions = None

    def fit(self, X_train, y_train):
        """
        Fit all three models
            
        :param X: train features
        :param y: train targets
        
        TODO: parallelize this code across processors
        """
        print("Calculating estimates...")
        self.mid_model.fit(X_train, y_train)
        print("Getting lower bound...")
        self.lower_model.fit(X_train, y_train)
        print("Getting upper bound...")
        self.upper_model.fit(X_train, y_train)

    def predict(self, Z):
        predictions["lower"] = self.lower_model.predict(Z)
        predictions["mid"] = self.mid_model.predict(Z)
        predictions["upper"] = self.upper_model.predict(Z)
        self.predictions = predictions
    
    def predict_drug(self, X_drug, X_target, first_pass, temp = 35, pH = 7, org = None):
        """
        Predict with all 3 models 
        
        :param X: test features
        :param y: test targets
        :return predictions: dataframe of predictions
        
        TODO: parallelize this code across processors
        """
        df_data['SMILES'] = X_drug
        df_data['Target Sequence'] = X_target
        if self.temp_ph:
            df_data['pH'] = pH
            df_data['Temp'] = temp
        df_data=df_data.join(self.org_list)
        if org is not None:
            df_data['var_'+org] = 1
        
        df_data = feature_select(df_data, self.drug_func_list, self.prot_func_list)
        discard=['SMILES','Target Sequence','Organism','IC50','pIC50','ID','InChI','PubChem_ID','UniProt_ID','Kd','Ki','EC50','kon','koff']
        df_vars=df_data.columns.values.tolist()
        to_keep=[i for i in df_vars if i not in discard]
        X=df_data[to_keep]
        X[len(X.columns)] = first_pass
        Z = flattener(X)
        
        predictions = self.predict(Z)

        return predictions
    
    def plot_intervals(self, mid=False, start=None, stop=None):
        """
        Plot the prediction intervals
        
        :param mid: boolean for whether to show the mid prediction
        :param start: optional parameter for subsetting start of predictions
        :param stop: optional parameter for subsetting end of predictions
    
        :return fig: plotly figure
        """

        if self.predictions is None:
            raise ValueError("This model has not yet made predictions.")
            return
        
        fig = plot_intervals(predictions, mid=mid, start=start, stop=stop)
        return fig
    
    def calculate_and_show_errors(self):
        """
        Calculate and display the errors associated with a set of prediction intervals
        
        :return fig: plotly boxplot of absolute error metrics
        """
        if self.predictions is None:
            raise ValueError("This model has not yet made predictions.")
            return
        
        calculate_error(self.predictions)
        fig = show_metrics(self.predictions)
        return fig